In [ ]:
import pandas as pd
import requests
import sqlalchemy

## Create "airports" dataframe

In [ ]:
# coordinates for Berlin, Hamburg, Munich
# latitudes = [52.5200, 53.5511, 48.1374]
# longitudes = [13.4050, 9.9937, 11.5755]

def icao_airport_codes(latitudes, longitudes):

  #assert len(latitudes) == len(longitudes)

  list_for_df = []

  for i in range(len(latitudes)):

    url = "https://aerodatabox.p.rapidapi.com/airports/search/location"

    querystring = {"lat":latitudes[i],"lon":longitudes[i],"radiusKm":"50","limit":"5","withFlightInfoOnly":"true"}
    headers = {
      "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
      "X-RapidAPI-Key": "14a44098c8mshe4536a007985112p1e3b4bjsn8fd805eb6bd4" #USE THIS KEY
    }

    response = requests.request("GET", url, headers=headers, params=querystring)

    list_for_df.append(pd.json_normalize(response.json()['items']))

  return pd.concat(list_for_df, ignore_index=True)

In [ ]:
# coordinates for Berlin, Hamburg, Munich
latitudes = [52.5200, 53.5511, 48.1374]
longitudes = [13.4050, 9.9937, 11.5755]

icao_airport_codes_df = icao_airport_codes(latitudes, longitudes)
icao_airport_codes_df

,icao,iata,name,shortName,municipalityName,countryCode,timeZone,location.lat,location.lon
0,EDDB,BER,Berlin Brandenburg,Brandenburg,Berlin,DE,Europe/Berlin,52.35139,13.493889
1,EDDH,HAM,Hamburg,Hamburg,Hamburg,DE,Europe/Berlin,53.63040,9.988229
2,EDDM,MUC,Munich,Munich,Munich,DE,Europe/Berlin,48.35380,11.786100


## Prepare a dataframe for pushing to the database

In [ ]:
# Selecting only the columns we need
airports_to_db = icao_airport_codes_df[["icao", "name", "municipalityName"]]
airports_to_db

,icao,name,municipalityName
0,EDDB,Berlin Brandenburg,Berlin
1,EDDH,Hamburg,Hamburg
2,EDDM,Munich,Munich


### Read the cities table from the database to fetch the "city_id" column

In the same manner as with the "weather" dataframe the "airports" dataframe will be related to the "cities" table in the database. Thus, we need to get the corresponding city id from the cities table and implement it in the airports one (will become a foreign key here). Since the "airports" is a static table (it won't be updated on a regular basis wich means that the script for creating it will not be run on the cloud) we can get the city_ids there via merging it with the "cities" table.

In [ ]:
from Keys import MySQL_pass

schema = "gans_local" # The name of your database
host = "127.0.0.1"
user = "root"
password = "USE YOUR MySQL PASSWORD HERE" # Your MySQL password
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [ ]:
cities_df = pd.read_sql("cities", con=connection_string)
cities_df

,City_id,City_name,Country
0,1,Berlin,Germany
1,2,Hamburg,Germany
2,3,Munich,Germany


#### Getting the "City_id" to the airports_to_db dataframe

In [ ]:
cities_airports_merged = cities_df.merge(airports_to_db,
                                   left_on = "City_name",
                                   right_on = "municipalityName",
                                   how="left")

cities_airports_merged

,City_id,City_name,Country,icao,name,municipalityName
0,1,Berlin,Germany,EDDB,Berlin Brandenburg,Berlin
1,2,Hamburg,Germany,EDDH,Hamburg,Hamburg
2,3,Munich,Germany,EDDM,Munich,Munich


In [ ]:
# Selecting only the columns we need
airports_to_db = cities_airports_merged[["icao","name", "City_id"]]

In [ ]:
airports_to_db

,icao,name,City_id
0,EDDB,Berlin Brandenburg,1
1,EDDH,Hamburg,2
2,EDDM,Munich,3


In [ ]:
airports_to_db.rename(columns={"name": "Airport_name"}, inplace=True)

/var/folders/44/00l5g0t953g1bjr6xszqqd400000gn/T/ipykernel_25945/2246797738.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  airports_to_db.rename(columns={"name": "Airport_name"}, inplace=True)


In [ ]:
airports_to_db.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3 entries, 0 to 2
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   icao          3 non-null      object
 1   Airport_name  3 non-null      object
 2   City_id       3 non-null      int64 
dtypes: int64(1), object(2)
memory usage: 96.0+ bytes


### Create "airports" table in the database

```sql
CREATE TABLE airports(
    icao VARCHAR(10),
    Airport_name VARCHAR(255),
    City_id INT NOT NULL,
    PRIMARY KEY (icao),
    FOREIGN KEY (City_id) REFERENCES cities(City_id)
);
```

### Push the "airports_to_db" to the empty "airports" table in the database

In [ ]:
airports_to_db.to_sql('airports',
                  if_exists='append',
                  con=connection_string,
                  index=False)

3

In [ ]:
pd.read_sql("airports", con=connection_string)

,icao,Airport_name,City_id
0,EDDB,Berlin Brandenburg,1
1,EDDH,Hamburg,2
2,EDDM,Munich,3
